In [93]:
import Pkg
Pkg.add(["Statistics","StatsBase","Plots","StatsPlots","KernelDensity","MLBase","HypothesisTests","DataFrames"]) 

   Resolving package versions...
  No Changes to `C:\Users\Aleyna\.julia\environments\v1.9\Project.toml`
  No Changes to `C:\Users\Aleyna\.julia\environments\v1.9\Manifest.toml`


In [ ]:
using CSV, Statistics, StatsBase, Plots, StatsPlots, KernelDensity, MLBase, HypothesisTests, DataFrames

In [55]:
using DataFrames


# CSV dosyasını oku
df = DataFrame(CSV.File("yenitemiz_data.csv"))

Row,VoteCount,Rating,Title,Season_Count,Episode_Count,Actors,cat_drama,cat_family,cat_action,cat_comedy,cat_romance,cat_adventure,cat_crime,cat_thriller,cat_horror,cat_mystery,cat_war,cat_fantasy,cat_sci-fi,cat_music,cat_biography,cat_documentary,cat_news,cat_sport,cat_animation,cat_reality-tv,cat_talk-show,cat_game-show,Release_Year,IsEnded,Classification_Int
,Float64,Float64,String,Int64,Int64,String?,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,105.0,7.2,Adalett,1,10,"Yurdaer Okur, Burak Sergen, Erdem Kaynarca",0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2023,0,1
2,12.0,4.5,%100 Futbol,1,9,"Ridvan Dilmen, Erbatur Ergenekon, Murat Kosova",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2005,0,1
3,454.0,7.7,1 Erkek 1 Kadin 2 Cocuk,1,33,"Demet Evgar, Emre Karayel",0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2014,1,1
4,5700.0,7.8,1 Kadın 1 Erkek,8,435,"Demet Evgar, Emre Karayel, Deniz Degirmenci",0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2008,1,4
5,2200.0,7.8,10 Bin Adim,2,20,"Engin Günaydin, Devin Özgür Çinar, Bugu Selen Somer",1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2020,0,2
6,70.0,8.0,103 Gol,1,4,"Toni Schumacher, Aykut Kocaman, Ridvan Dilmen",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,2021,1,1
7,0.0,5.66768,12 Eylül 1980 Darbe,1,4,"Onur Ayçelik, Aynur Alp, Mehmet Sarikaya",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,2010,1,1
8,0.0,5.66768,15 Dakika,1,3,"Yasar Duran, Fevzi Tuncay, Ciguli",0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,2017,1,1
9,12.0,3.4,2.Sayfa,5,460,"Müge Dagistanli, Gülsen Yüksel, Emir can Yurtlak",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,2010,0,3


In [56]:
# Sınıflandırmayı gerçekleştirin
df.Classification .= [
    typeof(year) === Int && season_count == 1 ? "A. Birinci sezonda yayından kaldırılacak mı? (Başarısız)" :
    season_count == 2 ? "B. Sadece iki sezon mu devam edecek? (Kısmen Başarılı)" :
    3 <= season_count <= 5 ? "C. Üç sezon ile beş sezon arasında mı sonlanacak? (Başarılı)" :
    5 < season_count <= 10 ? "D. Beş sezon ile on sezon arasında mı sonlanacak? (Kısmen Olgun)" :
    season_count > 10 ? "E. On sezon veya daha uzun süre devam edecek mi? (Olgun)" : "" for (year, season_count) in zip(df.IsEnded, df.Season_Count)
]


2187-element Vector{String}:
 "A. Birinci sezonda yayından kaldırılacak mı? (Başarısız)"
 "A. Birinci sezonda yayından kaldırılacak mı? (Başarısız)"
 "A. Birinci sezonda yayından kaldırılacak mı? (Başarısız)"
 "D. Beş sezon ile on sezon arasında mı sonlanacak? (Kısmen Olgun)"
 "B. Sadece iki sezon mu devam edecek? (Kısmen Başarılı)"
 "A. Birinci sezonda yayından kaldırılacak mı? (Başarısız)"
 "A. Birinci sezonda yayından kaldırılacak mı? (Başarısız)"
 "A. Birinci sezonda yayından kaldırılacak mı? (Başarısız)"
 "C. Üç sezon ile beş sezon arasında mı sonlanacak? (Başarılı)"
 "A. Birinci sezonda yayından kaldırılacak mı? (Başarısız)"
 "B. Sadece iki sezon mu devam edecek? (Kısmen Başarılı)"
 "B. Sadece iki sezon mu devam edecek? (Kısmen Başarılı)"
 "B. Sadece iki sezon mu devam edecek? (Kısmen Başarılı)"
 ⋮
 "A. Birinci sezonda yayından kaldırılacak mı? (Başarısız)"
 "B. Sadece iki sezon mu devam edecek? (Kısmen Başarılı)"
 "A. Birinci sezonda yayından kaldırılacak mı? (Başarısız)"
 "A. B

In [57]:
# Yeni bir sütun ekleyin ve başlangıçta boş bir dizi oluşturun
df.Classification_Int = Vector{Int64}(undef, size(df, 1))

# Her bir sınıflandırmayı integer bir değere dönüştür
for i in 1:size(df, 1)
    classification_str = df.Classification[i]

    df.Classification_Int[i] = 
        classification_str == "A. Birinci sezonda yayından kaldırılacak mı? (Başarısız)" ? 1 :
        classification_str == "B. Sadece iki sezon mu devam edecek? (Kısmen Başarılı)" ? 2 :
        classification_str == "C. Üç sezon ile beş sezon arasında mı sonlanacak? (Başarılı)" ? 3 :
        classification_str == "D. Beş sezon ile on sezon arasında mı sonlanacak? (Kısmen Olgun)" ? 4 :
        classification_str == "E. On sezon veya daha uzun süre devam edecek mi? (Olgun)" ? 5 :
        0  # Eğer sınıflandırma mevcut değilse veya hatalıysa 0 olarak işaretle
end


In [58]:
using DecisionTree
using Random

In [59]:
test_set = DataFrame(CSV.File("test_data.csv"))
train_set = DataFrame(CSV.File("train_data.csv"))

Row,VoteCount,Rating,Title,Episode_Count,cat_drama,cat_family,cat_action,cat_comedy,cat_romance,cat_adventure,cat_crime,cat_thriller,cat_horror,cat_mystery,cat_war,cat_fantasy,cat_sci-fi,cat_music,cat_biography,cat_documentary,cat_news,cat_sport,cat_animation,cat_reality-tv,cat_talk-show,cat_game-show,Release_Year,IsEnded,Classification_Int
,Float64,Float64,String,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,105.0,7.2,Adalett,10,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2023,0,1
2,12.0,4.5,%100 Futbol,9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2005,0,1
3,454.0,7.7,1 Erkek 1 Kadin 2 Cocuk,33,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2014,1,1
4,5700.0,7.8,1 Kadın 1 Erkek,435,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2008,1,4
5,2200.0,7.8,10 Bin Adim,20,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2020,0,2
6,70.0,8.0,103 Gol,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,2021,1,1
7,0.0,5.66768,15 Dakika,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,2017,1,1
8,12.0,3.4,2.Sayfa,460,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,2010,0,3
9,3000.0,7.6,20 Dakika,25,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2013,1,1


In [60]:
# Eğitim ve test setlerini gözlemleyin
@show size(train_set)
@show size(test_set)

size(train_set) = (1750, 29)
size(test_set) = (437, 26)


(437, 26)

In [61]:
# Kullanılan özellikler ve hedef değişken
features = select(train_set, Not(:Classification_Int, :Title))  # Title sütunu özellikler arasında kullanılmayacak
target = train_set[:, :Classification_Int]


1750-element Vector{Int64}:
 1
 1
 1
 4
 2
 1
 1
 3
 1
 2
 2
 2
 1
 ⋮
 3
 2
 1
 1
 1
 1
 2
 1
 1
 1
 3
 2

In [62]:
# Karar Ağacı modelini tanımlayın ve eğitin
model = DecisionTreeClassifier(rng=42)
DecisionTree.fit!(model, Matrix(features), target)

DecisionTreeClassifier
max_depth:                -1
min_samples_leaf:         1
min_samples_split:        2
min_purity_increase:      0.0
pruning_purity_threshold: 1.0
n_subfeatures:            0
classes:                  [1, 2, 3, 4, 5]
root:                     Decision Tree
Leaves: 349
Depth:  27

In [63]:
# Modelin performansını eğitim setinde değerlendirin
train_predictions = DecisionTree.predict(model, Matrix(features))
accuracy_train = mean(target .== train_predictions)
@show accuracy_train

accuracy_train = 0.9988571428571429


0.9988571428571429

In [66]:
# Test seti üzerinde modelin tahminlerini yap
test_features = select(test_set, Not(:Title))
test_predictions = DecisionTree.predict(model, Matrix(test_features))

# Modelin tahminlerini test setine ekleyerek yeni bir DataFrame oluştur
test_results = hcat(test_set, DataFrame(Predicted_Classification_Int = test_predictions))


Row,VoteCount,Rating,Title,cat_drama,cat_family,cat_action,cat_comedy,cat_romance,cat_adventure,cat_crime,cat_thriller,cat_horror,cat_mystery,cat_war,cat_fantasy,cat_sci-fi,cat_music,cat_biography,cat_documentary,cat_news,cat_sport,cat_animation,cat_reality-tv,cat_talk-show,cat_game-show,Release_Year,Predicted_Classification_Int
,Float64,Float64,String,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,3200.0,6.2,Kaçış,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2022,1
2,27.0,6.6,Harbi Getto,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2019,1
3,9600.0,7.1,Medcezir,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,2013,1
4,87.0,7.3,Rabarba Talk,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2020,1
5,14.0,2.8,Dicle,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,2007,1
6,53.0,7.1,Mahsusa: Trablusgarb,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2023,1
7,659.0,6.0,Bodrum Masalı,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2016,1
8,196.0,5.0,Adim Basi Kafe,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2021,1
9,0.0,5.66768,Müjdat Gezen'le Pazar Soleni,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,2002,1


In [ ]:
using DataFrames


# CSV dosyasını oku
df = DataFrame(CSV.File("yeni_data.csv"))